Underfitting
- training dataset에 대한 정확도가 매우 낮음
- test dataset 정확도 낮음

Overfitting
- training dataset에 대한 정확도가 매우 높음
- test dataset 정확도 낮음: training dataset에만 맞춰져 있어 새로운 데이터에 대한 정확도가 낮아짐

Dropout
- underfitting, overfitting을 방지하는 regularization 기법
- 학습 과정에서 전체 neuron(node) 중 일부만 사용하는 방식
- 어떤 node를 사용할지는 random
- test시에는 전체 node 사용
- 학습 과정에서 데이터의 전체를 보는 것이 아닌 부분을 보고 예측하는 방식

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os

tf.enable_eager_execution()

In [2]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

Network 형성을 위한 함수

In [3]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def relu() :
    return tf.keras.layers.Activation(tf.keras.activations.relu)

# rate: 전체 node 중 몇퍼센트의 node를 사용하지 않을 것인지(0~1 사이 값)
def dropout(rate) :
    return tf.keras.layers.Dropout(rate)

In [4]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.glorot_uniform()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(4) :
        model.add(dense(512, weight_init))
        model.add(relu())
        # rate=0.5: 전체 node의 50%를 제외한 상태로 학습하겠다
        model.add(dropout(rate=0.5))

    model.add(dense(label_dim, weight_init))

    return model

In [5]:
def loss_fn(model, images, labels):
    # training=True: dropout을 사용하겠다(training)
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels))
    return loss

def accuracy_fn(model, images, labels):
    # training=False: dropout을 사용하지 않겠다(test)
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

In [6]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size).\
    repeat()

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x)).\
    repeat()

train_iterator = train_dataset.make_one_shot_iterator()
test_iterator = test_dataset.make_one_shot_iterator()

Instructions for updating:
Colocations handled automatically by placer.


In [7]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

In [8]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_dropout'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

In [ ]:

if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.contrib.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)
    global_step = tf.train.create_global_step()

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)
        start_iteration = checkpoint_counter - start_epoch * training_iterations
        counter = checkpoint_counter
        global_step.assign(checkpoint_counter)
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default(), tf.contrib.summary.always_record_summaries():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx in range(start_iteration, training_iterations):
                train_input, train_label = train_iterator.get_next()
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables), global_step=global_step)

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)

                test_input, test_label = test_iterator.get_next()
                test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.contrib.summary.scalar(name='train_loss', tensor=train_loss)
                tf.contrib.summary.scalar(name='train_accuracy', tensor=train_accuracy)
                tf.contrib.summary.scalar(name='test_accuracy', tensor=test_accuracy)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    test_input, test_label = test_iterator.get_next()
    test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 2.6538, train_loss: 2.33766532, train_accuracy: 0.1719, test_Accuracy: 0.1397
Epoch: [ 0] [    1/  468] time: 3.2964, train_loss: 2.31873655, train_accuracy: 0.1797, test_Accuracy: 0.1826
Epoch: [ 0] [    2/  468] time: 4.0206, train_loss: 2.34587860, train_accuracy: 0.2344, test_Accuracy: 0.2129
Epoch: [ 0] [    3/  468] time: 4.7496, train_loss: 2.32488203, train_accuracy: 0.2578, test_Accuracy: 0.2428
Epoch: [ 0] [    4/  468] time: 5.3378, train_loss: 2.24219561, train_accuracy: 0.2734, test_Accuracy: 0.2322
Epoch: [ 0] [    5/  468] time: 5.9830, train_loss: 2.30829167, train_accuracy: 0.3203, test_Accuracy: 0.2699
Epoch: [ 0] [    6/  46

Epoch: [ 0] [   71/  468] time: 48.6454, train_loss: 0.67970222, train_accuracy: 0.8672, test_Accuracy: 0.8745
Epoch: [ 0] [   72/  468] time: 49.5015, train_loss: 0.93400198, train_accuracy: 0.8203, test_Accuracy: 0.8784
Epoch: [ 0] [   73/  468] time: 51.2562, train_loss: 0.71351063, train_accuracy: 0.8359, test_Accuracy: 0.8825
Epoch: [ 0] [   74/  468] time: 53.3031, train_loss: 0.59183896, train_accuracy: 0.8438, test_Accuracy: 0.8841
Epoch: [ 0] [   75/  468] time: 54.0433, train_loss: 0.63432932, train_accuracy: 0.8750, test_Accuracy: 0.8855
Epoch: [ 0] [   76/  468] time: 54.6574, train_loss: 0.65279531, train_accuracy: 0.8750, test_Accuracy: 0.8816
Epoch: [ 0] [   77/  468] time: 55.3108, train_loss: 0.58662313, train_accuracy: 0.8750, test_Accuracy: 0.8804
Epoch: [ 0] [   78/  468] time: 55.9797, train_loss: 0.72188497, train_accuracy: 0.8594, test_Accuracy: 0.8809
Epoch: [ 0] [   79/  468] time: 56.6177, train_loss: 0.62104976, train_accuracy: 0.8828, test_Accuracy: 0.8839
E

Epoch: [ 0] [  145/  468] time: 102.0996, train_loss: 0.25185323, train_accuracy: 0.9531, test_Accuracy: 0.9255
Epoch: [ 0] [  146/  468] time: 102.7562, train_loss: 0.56244648, train_accuracy: 0.8984, test_Accuracy: 0.9231
Epoch: [ 0] [  147/  468] time: 103.5048, train_loss: 0.36134106, train_accuracy: 0.9375, test_Accuracy: 0.9222
Epoch: [ 0] [  148/  468] time: 104.2082, train_loss: 0.53885889, train_accuracy: 0.8750, test_Accuracy: 0.9221
Epoch: [ 0] [  149/  468] time: 104.8709, train_loss: 0.38546199, train_accuracy: 0.9062, test_Accuracy: 0.9252
Epoch: [ 0] [  150/  468] time: 105.5808, train_loss: 0.37061921, train_accuracy: 0.9375, test_Accuracy: 0.9271
Epoch: [ 0] [  151/  468] time: 106.2543, train_loss: 0.37936312, train_accuracy: 0.9609, test_Accuracy: 0.9273
Epoch: [ 0] [  152/  468] time: 106.9015, train_loss: 0.22602159, train_accuracy: 0.9531, test_Accuracy: 0.9280
Epoch: [ 0] [  153/  468] time: 107.5218, train_loss: 0.36985293, train_accuracy: 0.9531, test_Accuracy:

Epoch: [ 0] [  219/  468] time: 173.7571, train_loss: 0.49314255, train_accuracy: 0.9141, test_Accuracy: 0.9324
Epoch: [ 0] [  220/  468] time: 174.3868, train_loss: 0.27541628, train_accuracy: 0.9844, test_Accuracy: 0.9352
Epoch: [ 0] [  221/  468] time: 174.9531, train_loss: 0.34889179, train_accuracy: 0.9375, test_Accuracy: 0.9363
Epoch: [ 0] [  222/  468] time: 175.6363, train_loss: 0.32854441, train_accuracy: 0.9609, test_Accuracy: 0.9347
Epoch: [ 0] [  223/  468] time: 176.2629, train_loss: 0.46867359, train_accuracy: 0.9297, test_Accuracy: 0.9332
Epoch: [ 0] [  224/  468] time: 176.9402, train_loss: 0.39175430, train_accuracy: 0.9531, test_Accuracy: 0.9321
Epoch: [ 0] [  225/  468] time: 177.5779, train_loss: 0.30898678, train_accuracy: 0.9531, test_Accuracy: 0.9340
Epoch: [ 0] [  226/  468] time: 178.1999, train_loss: 0.26700586, train_accuracy: 0.9766, test_Accuracy: 0.9345
Epoch: [ 0] [  227/  468] time: 179.0562, train_loss: 0.41968900, train_accuracy: 0.9375, test_Accuracy:

Epoch: [ 0] [  293/  468] time: 222.5368, train_loss: 0.26121259, train_accuracy: 0.9219, test_Accuracy: 0.9481
Epoch: [ 0] [  294/  468] time: 223.3442, train_loss: 0.39301684, train_accuracy: 0.9375, test_Accuracy: 0.9482
Epoch: [ 0] [  295/  468] time: 224.0212, train_loss: 0.42911643, train_accuracy: 0.9062, test_Accuracy: 0.9475
Epoch: [ 0] [  296/  468] time: 224.6732, train_loss: 0.17614073, train_accuracy: 0.9922, test_Accuracy: 0.9472
Epoch: [ 0] [  297/  468] time: 225.2846, train_loss: 0.35919330, train_accuracy: 0.9453, test_Accuracy: 0.9469
Epoch: [ 0] [  298/  468] time: 226.0490, train_loss: 0.20591262, train_accuracy: 0.9688, test_Accuracy: 0.9464
Epoch: [ 0] [  299/  468] time: 227.1223, train_loss: 0.20437330, train_accuracy: 0.9688, test_Accuracy: 0.9460
Epoch: [ 0] [  300/  468] time: 228.1576, train_loss: 0.18200508, train_accuracy: 0.9609, test_Accuracy: 0.9460
Epoch: [ 0] [  301/  468] time: 229.1240, train_loss: 0.16509730, train_accuracy: 0.9609, test_Accuracy:

Epoch: [ 0] [  367/  468] time: 272.5820, train_loss: 0.50324523, train_accuracy: 0.9062, test_Accuracy: 0.9534
Epoch: [ 0] [  368/  468] time: 273.1493, train_loss: 0.23424509, train_accuracy: 0.9688, test_Accuracy: 0.9529
Epoch: [ 0] [  369/  468] time: 273.8062, train_loss: 0.27072826, train_accuracy: 0.9531, test_Accuracy: 0.9530
Epoch: [ 0] [  370/  468] time: 274.4976, train_loss: 0.17461637, train_accuracy: 0.9766, test_Accuracy: 0.9532
Epoch: [ 0] [  371/  468] time: 275.2421, train_loss: 0.21392342, train_accuracy: 0.9375, test_Accuracy: 0.9530
Epoch: [ 0] [  372/  468] time: 275.8969, train_loss: 0.17198786, train_accuracy: 0.9531, test_Accuracy: 0.9532
Epoch: [ 0] [  373/  468] time: 277.1440, train_loss: 0.27320528, train_accuracy: 0.9453, test_Accuracy: 0.9533
Epoch: [ 0] [  374/  468] time: 278.1519, train_loss: 0.29834706, train_accuracy: 0.9609, test_Accuracy: 0.9527
Epoch: [ 0] [  375/  468] time: 279.0274, train_loss: 0.38954979, train_accuracy: 0.9219, test_Accuracy:

Epoch: [ 0] [  441/  468] time: 323.8028, train_loss: 0.21603167, train_accuracy: 0.9531, test_Accuracy: 0.9546
Epoch: [ 0] [  442/  468] time: 324.5592, train_loss: 0.29935592, train_accuracy: 0.9688, test_Accuracy: 0.9557
Epoch: [ 0] [  443/  468] time: 325.6584, train_loss: 0.36131722, train_accuracy: 0.9141, test_Accuracy: 0.9564
Epoch: [ 0] [  444/  468] time: 326.3408, train_loss: 0.42427498, train_accuracy: 0.9688, test_Accuracy: 0.9571
Epoch: [ 0] [  445/  468] time: 327.0076, train_loss: 0.29649854, train_accuracy: 0.9531, test_Accuracy: 0.9578
Epoch: [ 0] [  446/  468] time: 327.6890, train_loss: 0.21680722, train_accuracy: 0.9531, test_Accuracy: 0.9585
Epoch: [ 0] [  447/  468] time: 328.4162, train_loss: 0.17748885, train_accuracy: 0.9766, test_Accuracy: 0.9573
Epoch: [ 0] [  448/  468] time: 329.3177, train_loss: 0.33477777, train_accuracy: 0.9453, test_Accuracy: 0.9570
Epoch: [ 0] [  449/  468] time: 330.3353, train_loss: 0.21444918, train_accuracy: 0.9531, test_Accuracy: